In [4]:
!pip install scikit-learn

  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl (11.1 MB)
   ---------------------------------------- 0.0/40.9 MB ? eta -:--:--
   - -------------------------------------- 1.3/40.9 MB 8.4 MB/s eta 0:00:05
   --- ------------------------------------ 3.9/40.9 MB 10.7 MB/s eta 0:00:04
   ------- -------------------------------- 7.3/40.9 MB 12.6 MB/s eta 0:00:03
   --------- ------------------------------ 9.7/40.9 MB 12.6 MB/s eta 0:00:03
   ----------- ---------------------------- 11.3/40.9 MB 11.4 MB/s eta 0:00:03
   -------------- ------------------------- 14.7/40.9 MB 12.1 MB/s eta 0:00:03
   ------------------ --------------------- 18.6/40.9 MB 13.2 MB/s eta 0:00:02
   ---------------------- ----------------- 22.8/40.9 MB 14.1 MB/s eta 0:00:02
   -------------------------- ------------- 26.7/40.9 MB 14.6 MB/s eta 0:00:01
   ------------------------------ --------- 30.9/40.9 MB 15.2 MB/s eta 0:00:01



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install datasets

  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   -- ------------------------------------- 1.8/25.3 MB 10.1 MB/s eta 0:00:03
   ----- ---------------------------------- 3.7/25.3 MB 9.1 MB/s eta 0:00:03
   ---------- ----------------------------- 6.6/25.3 MB 10.9 MB/s eta 0:00:02
   --------------- ------------------------ 10.0/25.3 MB 12.4 MB/s eta 0:00:02
   --------------------- ------------------ 13.4/25.3 MB 13.1 MB/s eta 0:00:01
   --------------------------- ------------ 17.3/25.3 MB 14.2 MB/s eta 0:00:01
   --------------------------------- ------ 21.2/25.3 MB 14.9 MB/s eta 0:00:01
   ---------------------------------------  24.9/25.3 MB 15.3 MB/s eta 0:00:01
   ---------------------------------------- 25.3/25.3 MB 14.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fssp


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from datasets import Dataset

Load Dataset

In [9]:
data_path = "../data/jutsus.jsonl"
df = pd.read_json(data_path, lines=True)
df.head()

,jutsu_name,jutsu_type,jutsu_description
0,Adamantine Sealing Chains: Spiral Formation,"Hiden, Ninjutsu, Fūinjutsu, Barrier Ninjutsu, ...",Kushina uses her chains to form a barrier whil...
1,Adamantine Power: Acala,"Kekkei Genkai, Ninjutsu, Taijutsu",Hashirama kicks the opponent away and raises s...
2,Adamantine Prison Wall,"Ninjutsu, Clone Techniques, Bukijutsu","After using Transformation: Adamantine Staff, ..."
3,Adamantine Seal: Monkey Yang Suppression,"Ninjutsu, Fūinjutsu, Cooperation Ninjutsu","After placing fūinjutsu tags in an area, the u..."
4,Acrobat,"Taijutsu, Kenjutsu","The Acrobat (荒繰鷺伐刀, Akurobatto) is a kenjutsu ..."


In [10]:
def simplify_jutsu(jutsu):
    if "Genjutsu" in jutsu:
        return "Genjutsu"
    if "Ninjutsu" in jutsu:
        return "Ninjutsu"
    if "Taijutsu" in jutsu:
        return "Taijutsu"

In [11]:
df['jutsu_type_simplified'] = df['jutsu_type'].apply(simplify_jutsu)

In [12]:
df.head()

,jutsu_name,jutsu_type,jutsu_description,jutsu_type_simplified
0,Adamantine Sealing Chains: Spiral Formation,"Hiden, Ninjutsu, Fūinjutsu, Barrier Ninjutsu, ...",Kushina uses her chains to form a barrier whil...,Ninjutsu
1,Adamantine Power: Acala,"Kekkei Genkai, Ninjutsu, Taijutsu",Hashirama kicks the opponent away and raises s...,Ninjutsu
2,Adamantine Prison Wall,"Ninjutsu, Clone Techniques, Bukijutsu","After using Transformation: Adamantine Staff, ...",Ninjutsu
3,Adamantine Seal: Monkey Yang Suppression,"Ninjutsu, Fūinjutsu, Cooperation Ninjutsu","After placing fūinjutsu tags in an area, the u...",Ninjutsu
4,Acrobat,"Taijutsu, Kenjutsu","The Acrobat (荒繰鷺伐刀, Akurobatto) is a kenjutsu ...",Taijutsu


In [13]:
df['jutsu_type_simplified'].value_counts()

jutsu_type_simplified
Ninjutsu    2255
Taijutsu     397
Genjutsu     101
Name: count, dtype: int64

In [14]:
df['text'] = df['jutsu_name'] + ". " + df['jutsu_description']
df['jutsus'] = df['jutsu_type_simplified']
df = df[['text', 'jutsus']]
df = df.dropna()

In [15]:
df.head()

,text,jutsus
0,Adamantine Sealing Chains: Spiral Formation. K...,Ninjutsu
1,Adamantine Power: Acala. Hashirama kicks the o...,Ninjutsu
2,Adamantine Prison Wall. After using Transforma...,Ninjutsu
3,Adamantine Seal: Monkey Yang Suppression. Afte...,Ninjutsu
4,"Acrobat. The Acrobat (荒繰鷺伐刀, Akurobatto) is a ...",Taijutsu


In [16]:
from bs4 import BeautifulSoup
class Cleaner():
    def __init__(self):
        pass 
    
    def put_line_breaks(self, text):
        return text.replace("<\p>", "<\p>\n")
    
    def remove_html_tags(self, text):
        clean_text = BeautifulSoup(text, "lxml").text
        return clean_text

    def clean(self, text):
        text = self.put_line_breaks(text)
        text = self.remove_html_tags(text)
        text = text.strip()
        return text

<>:7: SyntaxWarning: invalid escape sequence '\p'
<>:7: SyntaxWarning: invalid escape sequence '\p'
<>:7: SyntaxWarning: invalid escape sequence '\p'
<>:7: SyntaxWarning: invalid escape sequence '\p'
C:\Users\anubh\AppData\Local\Temp\ipykernel_9792\3655626712.py:7: SyntaxWarning: invalid escape sequence '\p'
  return text.replace("<\p>", "<\p>\n")
C:\Users\anubh\AppData\Local\Temp\ipykernel_9792\3655626712.py:7: SyntaxWarning: invalid escape sequence '\p'
  return text.replace("<\p>", "<\p>\n")


In [17]:
text_column_name = 'text'
label_column_name = "jutsus"

In [18]:
# Clean Text
cleaner = Cleaner()
df['text_cleaned'] = df[text_column_name].apply(cleaner.clean)

In [19]:
df.head(2)

,text,jutsus,text_cleaned
0,Adamantine Sealing Chains: Spiral Formation. K...,Ninjutsu,Adamantine Sealing Chains: Spiral Formation. K...
1,Adamantine Power: Acala. Hashirama kicks the o...,Ninjutsu,Adamantine Power: Acala. Hashirama kicks the o...


In [20]:
# Encode Labels 
le = preprocessing.LabelEncoder()
le.fit(df[label_column_name].tolist())

LabelEncoder()

In [21]:
label_dict = {index:label_name for index, label_name in enumerate(le.__dict__['classes_'].tolist())}
label_dict

{0: 'Genjutsu', 1: 'Ninjutsu', 2: 'Taijutsu'}

In [22]:
df['label'] = le.transform(df[label_column_name].tolist())

In [23]:
df.head()

,text,jutsus,text_cleaned,label
0,Adamantine Sealing Chains: Spiral Formation. K...,Ninjutsu,Adamantine Sealing Chains: Spiral Formation. K...,1
1,Adamantine Power: Acala. Hashirama kicks the o...,Ninjutsu,Adamantine Power: Acala. Hashirama kicks the o...,1
2,Adamantine Prison Wall. After using Transforma...,Ninjutsu,Adamantine Prison Wall. After using Transforma...,1
3,Adamantine Seal: Monkey Yang Suppression. Afte...,Ninjutsu,Adamantine Seal: Monkey Yang Suppression. Afte...,1
4,"Acrobat. The Acrobat (荒繰鷺伐刀, Akurobatto) is a ...",Taijutsu,"Acrobat. The Acrobat (荒繰鷺伐刀, Akurobatto) is a ...",2


In [24]:
test_size = 0.2
df_train, df_test = train_test_split(df, 
                                     test_size=test_size, 
                                     stratify=df['label'],)

In [25]:
df_train['jutsus'].value_counts()

jutsus
Ninjutsu    1804
Taijutsu     317
Genjutsu      81
Name: count, dtype: int64

In [26]:
model_name = "distilbert/distilbert-base-uncased"

In [27]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

c:\Users\anubh\AppData\Local\Programs\Python\Python313\Scripts\Learning\TV Series Analysis System\env\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anubh\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [28]:
def preprocess_function(tokenizer,examples):
    return tokenizer(examples['text_cleaned'],truncation=True)

In [29]:
# Conver Pandas to a hugging face dataset
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

# tokenize the dataset
tokenized_train = train_dataset.map(lambda examples: preprocess_function(tokenizer, examples),
                                    batched=True)
tokenized_test = test_dataset.map(lambda examples: preprocess_function(tokenizer, examples),
                                    batched=True)

Map: 100%|██████████| 551/551 [00:00<00:00, 13526.85 examples/s]
